# Maestria en Explotacion de datos y Descubrimiento de conocimiento
### Sistemas de información geografica
### Trabajo Practico N°2

# Clasificación por pixel usando deep learning

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../lib')

In [58]:
import pandas as pd
import numpy as np

import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

## Constantes globales

In [4]:
RESULT_PATH  = "../results"

## Funciones helper

In [59]:
result_path = lambda file: f'{RESULT_PATH}/{file}'

In [60]:
df = pd.read_csv(result_path('dataset.csv'))

In [61]:
df = df.loc[:, ~df.columns.isin(['GEOMETRY', 'Unnamed: 0', 'in1', 'ogc_fid', 'originfid'])]
df

,id,cultivo,band_0,band_1,band_2,band_3,band_4,band_5,band_6,band_7,...,band_56,band_57,band_58,band_59,band_60,band_61,band_62,band_63,band_64,band_65
0,3,MAIZ,0.294909,0.891577,1.836513,0.228713,-0.871737,0.207711,0.213346,0.647454,...,0.897865,1.882518,0.155672,-1.709989,0.126186,0.130338,0.653081,0.425929,0.459842,0.254514
1,1,SOJA,0.354500,0.924392,2.098372,0.248241,-1.050501,0.218901,0.227279,0.677250,...,0.900739,1.904153,0.164812,-1.628484,0.134796,0.139384,0.655666,0.438454,0.510339,0.264981
2,2,MAIZ,0.265749,0.875071,1.723864,0.195119,-0.992671,0.173785,0.178379,0.632874,...,0.971879,2.600671,0.117817,-2.904561,0.086983,0.089539,0.722274,0.806963,0.416432,0.623472
3,2,MAIZ,0.329759,0.910911,1.984000,0.243527,-0.953812,0.218026,0.225289,0.664879,...,0.901640,1.911017,0.101192,-2.570306,0.076131,0.078012,0.656477,0.442412,0.260500,0.268313
4,2,MAIZ,0.326524,0.909134,1.969667,0.233421,-1.020433,0.206807,0.213882,0.663262,...,0.891177,1.833652,0.100895,-2.466148,0.076450,0.078313,0.647099,0.397356,0.239041,0.231089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,4,SOJA,0.320158,0.905626,1.941860,0.247406,-0.867436,0.224708,0.231434,0.660079,...,0.942516,2.269772,0.248538,-1.220251,0.213973,0.223493,0.694168,0.639582,1.128600,0.449049
462,4,SOJA,0.330309,0.911213,1.986453,0.243959,-0.953364,0.218427,0.225716,0.665155,...,0.899488,1.894685,0.170689,-1.544769,0.140932,0.145754,0.654539,0.432982,0.535340,0.260393
463,4,SOJA,0.308651,0.899250,1.892896,0.239855,-0.861503,0.218132,0.224330,0.654326,...,0.919030,2.051645,0.229376,-1.157066,0.199143,0.206764,0.672308,0.521891,0.946682,0.337742
464,4,SOJA,0.334020,0.913247,2.003094,0.246778,-0.951130,0.221018,0.228484,0.667010,...,0.975951,2.652838,0.226813,-1.699304,0.184730,0.194024,0.726240,0.832270,1.003968,0.651349


In [62]:
df_bands = df.loc[:, ~df.columns.isin(['id', 'cultivo'])]

for band in df_bands.columns:
    df[f'{band}_mean'] = np.repeat(df[band].mean(), df_bands.shape[0])

In [63]:
for band in df_bands.columns:
    df[f'{band}_var'] = np.repeat(df[band].var(), df_bands.shape[0])

In [64]:
df

,id,cultivo,band_0,band_1,band_2,band_3,band_4,band_5,band_6,band_7,...,band_56_var,band_57_var,band_58_var,band_59_var,band_60_var,band_61_var,band_62_var,band_63_var,band_64_var,band_65_var
0,3,MAIZ,0.294909,0.891577,1.836513,0.228713,-0.871737,0.207711,0.213346,0.647454,...,0.002644,0.401879,0.002519,0.38153,0.002171,0.002433,0.002386,0.092527,0.093637,0.109038
1,1,SOJA,0.354500,0.924392,2.098372,0.248241,-1.050501,0.218901,0.227279,0.677250,...,0.002644,0.401879,0.002519,0.38153,0.002171,0.002433,0.002386,0.092527,0.093637,0.109038
2,2,MAIZ,0.265749,0.875071,1.723864,0.195119,-0.992671,0.173785,0.178379,0.632874,...,0.002644,0.401879,0.002519,0.38153,0.002171,0.002433,0.002386,0.092527,0.093637,0.109038
3,2,MAIZ,0.329759,0.910911,1.984000,0.243527,-0.953812,0.218026,0.225289,0.664879,...,0.002644,0.401879,0.002519,0.38153,0.002171,0.002433,0.002386,0.092527,0.093637,0.109038
4,2,MAIZ,0.326524,0.909134,1.969667,0.233421,-1.020433,0.206807,0.213882,0.663262,...,0.002644,0.401879,0.002519,0.38153,0.002171,0.002433,0.002386,0.092527,0.093637,0.109038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461,4,SOJA,0.320158,0.905626,1.941860,0.247406,-0.867436,0.224708,0.231434,0.660079,...,0.002644,0.401879,0.002519,0.38153,0.002171,0.002433,0.002386,0.092527,0.093637,0.109038
462,4,SOJA,0.330309,0.911213,1.986453,0.243959,-0.953364,0.218427,0.225716,0.665155,...,0.002644,0.401879,0.002519,0.38153,0.002171,0.002433,0.002386,0.092527,0.093637,0.109038
463,4,SOJA,0.308651,0.899250,1.892896,0.239855,-0.861503,0.218132,0.224330,0.654326,...,0.002644,0.401879,0.002519,0.38153,0.002171,0.002433,0.002386,0.092527,0.093637,0.109038
464,4,SOJA,0.334020,0.913247,2.003094,0.246778,-0.951130,0.221018,0.228484,0.667010,...,0.002644,0.401879,0.002519,0.38153,0.002171,0.002433,0.002386,0.092527,0.093637,0.109038


In [65]:
df.to_csv(result_path('dataset_mean_var.csv'), encoding='utf-8')